<a href="https://colab.research.google.com/github/igrodfer/DeepLearningLab/blob/main/cifar10_8300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from __future__ import print_function
%load_ext autoreload
%autoreload 2
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils

from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
batch_size = 100
num_classes = 10
epochs = 75

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 13s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [5]:
## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(model,filters,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))
  
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  
  return model

In [17]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rotation_range=10)

In [30]:
## DEF NN TOPOLOGY  
model = Sequential()

model=CBGN(model,32,x_train.shape[1:])
model=CBGN(model,64)
model=CBGN(model,128)
model=CBGN(model,256)
model=CBGN(model,512)

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [31]:
## OPTIM AND COMPILE
opt = SGD(learning_rate=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [32]:
history=model.fit(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            verbose=1)

## TEST
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/75
500/500 [==============================] - 32s 61ms/step - loss: 1.7447 - accuracy: 0.3698 - val_loss: 1.6890 - val_accuracy: 0.4182
Epoch 2/75
500/500 [==============================] - 30s 60ms/step - loss: 1.3901 - accuracy: 0.4909 - val_loss: 1.4060 - val_accuracy: 0.5211
Epoch 3/75
500/500 [==============================] - 29s 59ms/step - loss: 1.2276 - accuracy: 0.5566 - val_loss: 1.1945 - val_accuracy: 0.5782
Epoch 4/75
500/500 [==============================] - 29s 59ms/step - loss: 1.1263 - accuracy: 0.5963 - val_loss: 1.7805 - val_accuracy: 0.4847
Epoch 5/75
500/500 [==============================] - 30s 60ms/step - loss: 1.0432 - accuracy: 0.6265 - val_loss: 1.3166 - val_accuracy: 0.5541
Epoch 6/75
500/500 [==============================] - 29s 58ms/step - loss: 0.9800 - accuracy: 0.6504 - val_loss: 1.1346 - val_accuracy: 0.6218
Epoch 7/75
500/500 [==============================] - 29s 59ms/step - loss: 0.9302 - accuracy: 0.6706 - val_loss: 1.2548 - val_accuracy: